# Lossy medium with resistivity 
### (σ - conductivity)
### sin pulse

In [1]:
import numpy as np
from bokeh.plotting import figure, show, gridplot, output_notebook

output_notebook()

Loading BokehJS ...

In [2]:
ke = 200

Ex = np.zeros(ke)
Hy = np.zeros(ke)

ddx = 0.01  # cell size
dt = ddx / 6e8  # time step size
freq_in = 700e6  # MHz


t0 = 40
spread = 12

bc_low = [0, 0]
bc_high = [0, 0]

# ε - dielectric profile
cb = 0.5 * np.ones(ke)
cb_start = 100
eps = 4
cb[cb_start:] = 0.5 / eps

nsteps = 425

Ex_hist = np.full([nsteps, ke], np.nan)
Hy_hist = np.full([nsteps, ke], np.nan)

##########################################################
for ts in range(1, nsteps + 1):
    # Ex
    for k in range(1, ke):
        Ex[k] = Ex[k] + cb[k] * (Hy[k - 1] - Hy[k])

    # SIN pulse
    pulse = np.sin(2 * np.pi * freq_in * dt * ts)
    Ex[5] = pulse + Ex[5]

    # Absorbing BC
    Ex[0] = bc_low.pop(0)
    bc_low.append(Ex[1])
    Ex[ke - 1] = bc_high.pop(0)
    bc_high.append(Ex[ke - 2])

    # Hy
    for k in range(ke - 1):
        Hy[k] = Hy[k] + 0.5 * (Ex[k] - Ex[k + 1])

    # Hist
    Ex_hist[ts - 1, :] = Ex
    Hy_hist[ts - 1, :] = Hy

In [3]:
_fparams = dict(width=900, height=300, x_axis_label="Z", y_axis_label="t")
fig1 = figure(**_fparams)
fig2 = figure(**_fparams)

_params = dict(level="image", x=0, y=0, dw=ke, dh=nsteps, palette="Turbo256")

fig1.image(image=[Ex_hist], **_params)
fig2.image(image=[Hy_hist], **_params)

show(gridplot([[fig1], [fig2]]))